In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [28]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
# cfg = Config.fromfile('./77_cascade_swins_fpn_3x_TTA.py')
# cfg = Config.fromfile('./2ed/85_cascade_swins_fpn_3x_iou_change.py')
# cfg = Config.fromfile('./myconfig.py')
cfg = Config.fromfile('./faster_rcnn_swin_pafpn_3x_coco.py')

# root='../../dataset/'
root='../../dataset_erase2/dataset/'
# epoch = 'latest'
epoch = 'best_bbox_mAP_50_epoch_24'
# epoch = 'epoch_24'
# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1

cfg.seed=2021
cfg.gpu_ids = [1]
# cfg.work_dir = './work_dirs/2_cascade_swins_fpn_adamW_resize1024'
# cfg.work_dir = './work_dirs/1_cascade_swins_fpn_adamW'
# cfg.work_dir = './work_dirs/carscade_rcnn_pvt2_pafpn_1x_trash/Adamw_step_aug2_iou'
# cfg.work_dir = './work_dirs/77_cascade_swins_fpn_3x_TTA_2'
# cfg.work_dir = './work_dirs/77_cascade_swins_fpn_3x_erase'
# cfg.work_dir = './work_dirs/77_cascade_swins_fpn_3x_erase_erase'
# cfg.work_dir = './work_dirs/77_cascade_swins_fpn_3x_erase_normal'
# cfg.work_dir = './work_dirs/77_cascade_swins_fpn_3x_erase_erase_relabelling'
cfg.work_dir = './'

# cfg.work_dir = './work_dirs/3_soft_teacher_base_720'
# cfg.work_dir = '/opt/ml/detection/baseline/mmdetection/work_dirs/77_cascade_swins_fpn_3x_TTA/'
# detection/baseline/mmdetection/work_dirs/77_cascade_swins_fpn_3x_TTA/latest.pth
# cfg.work_dir = './work_dirs/dynamic/3_pseudo'

cfg.model.roi_head.bbox_head.num_classes = 10
# detection/baseline/mmdetection/work_dirs/77_cascade_swins_fpn_3x_TTA/latest.pths


# cfg.model.roi_head.bbox_head[0].num_classes = 10
# cfg.model.roi_head.bbox_head[1].num_classes = 10
# cfg.model.roi_head.bbox_head[2].num_classes = 10
# cfg.model.bbox_head.num_classes = 10
# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None



In [29]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [30]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')
# checkpoint_path = '/opt/ml/detection/baseline/mmdetection/work_dirs/77_cascade_swins_fpn_3x_TTA_2/latest.pth'

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./best_bbox_mAP_50_epoch_24.pth


In [31]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 3.0 task/s, elapsed: 1620s, ETA:     0s

In [33]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
# submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}_.csv'), index=None)
submission.to_csv(os.path.join(cfg.work_dir, f'submission_faster_rcnn_swin_pafpn_3x_coco__best_bbox_mAP_50_epoch_24.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.11600642 481.15704 589.3275 572.985 660.14...,test/0000.jpg
1,0 0.41900325 633.676 647.79987 688.04517 845.2...,test/0001.jpg
2,0 0.9189069 878.9129 463.2321 1021.94037 630.8...,test/0002.jpg
3,0 0.06676256 43.975418 193.06616 980.4905 909....,test/0003.jpg
4,0 0.7281811 441.4682 394.5675 657.6543 567.054...,test/0004.jpg
